### Some initial preparations

We install the brainscapes library, and set an EBRAINS Knowledge Graph authentication token the we can request  [here](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize) after creating an HBP account.

In [ ]:
!pip install brainscapes
from os import environ
environ['HBP_AUTH_TOKEN'] = "your auth token here"

### 1. Instantiate human brain atlas with Julich-Brain cytoarchitectonic maps

We instantiate the human atlas, and tell it to prefer thresholded continous maps over labelled regions in the discrete parcellation map. For the Julich-Brain parcellation, this mean that it will use thresholded probability maps, if available.

In [1]:
import brainscapes as bs
bs.logger.setLevel("INFO") # we want to see some messages!

In [2]:
atlas = bs.atlases.MULTILEVEL_HUMAN_ATLAS
# next line is optional - cytoarchitectonic maps are selected by default
atlas.select_parcellation(bs.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_5_)
atlas.enable_continuous_map_thresholding(0.2)

[brainscapes:INFO]  Selected parcellation "Julich-Brain Probabilistic Cytoarchitectonic Maps (v2.5)"


### 2. Instantiate a differential gene expression analysis with this atlas

The analysis object will check if the parcellation selected in the atlas is suitable for the analysis. What it does in fact, is to verify that it provides maps in the MNI ICBM 152 space. 

We add a few candidate genes that we're interested in. They could also be added right away as a list to the constructor in the first line below.

Then we select two regions of interest (ROI) by specifying brain area names that the atlas can interpret.

In [7]:
jugex = bs.analysis.DifferentialGeneExpression(atlas)
jugex.add_candidate_gene(bs.features.gene_names.MAOA)
jugex.add_candidate_gene(bs.features.gene_names.TAC1)
jugex.define_roi1("v1 right")
jugex.define_roi2("v2 right")

[brainscapes:INFO]  Selected region Area hOc1 (V1, 17, CalcS) - right hemisphere
[brainscapes:INFO]  Retrieving probe ids for gene MAOA


Using continuous map thresholded by 0.2 for masking region Area hOc1 (V1, 17, CalcS) - right hemisphere.


[brainscapes:INFO]  Retrieving probe ids for gene TAC1
[brainscapes:INFO]  13 samples found for region v1 right.
[brainscapes:INFO]  Selected region Area hOc2 (V2, 18) - right hemisphere
[brainscapes:INFO]  Retrieving probe ids for gene MAOA


Using continuous map thresholded by 0.2 for masking region Area hOc2 (V2, 18) - right hemisphere.


[brainscapes:INFO]  Retrieving probe ids for gene TAC1
[brainscapes:INFO]  13 samples found for region v2 right.


### 3. Run the analysis

Now we actually run the analysis, providing a number of permutations.

In [8]:
jugex.run(permutations=1000)
print(jugex.result)

[brainscapes:INFO]  Running 1000 random permutations. This may take a while...


{'MAOA': 0.553, 'TAC1': 0.081}
